In [ ]:
import keras

In [ ]:
!mkdir images

In [ ]:
# List of no logo dataset
import os
files=os.listdir('/content/drive/MyDrive/DUNZO/nologo')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator,img_to_array,array_to_img,load_img


In [ ]:
# Image Data generator to create multiple images from less data
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')



In [ ]:
# Creating data augmentation and storing the images to increase the dataset size
# Storing the images folder wise so that it can be passed through ImageDataGenerator
for f in files:
  img = load_img('/content/drive/MyDrive/DUNZO/nologo/'+f)  
  x = img_to_array(img) 
  x = x.reshape((1,) + x.shape) 

  i = 0
  for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='/content/drive/MyDrive/DUNZO/nologo', save_prefix='dunzo', save_format='jpeg'):
    i += 1
    if i > 5:
        break   

In [ ]:
!unzip /content/drive/MyDrive/DUNZO.zip -d /content/

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))



In [ ]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 16

# this is the augmentation for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation for testing:
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        '/content/DUNZO/train',  
        target_size=(150, 150),  
        batch_size=batch_size,
        class_mode='categorical')  

# for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/DUNZO/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 1397 images belonging to 2 classes.
Found 341 images belonging to 2 classes.


In [ ]:
# Callbacks to save the best model and early stop to stop if accuracy does not increase
checkpoint_filepath = '/content/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
earlystop=keras.callbacks.EarlyStopping(monitor='val_accuracy',mode='max',patience=5,restore_best_weights=True)

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1397 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=341 // batch_size,
        callbacks=[model_checkpoint_callback,earlystop])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
60/87 [===================>..........] - ETA: 4s - loss: 0.4598 - accuracy: 0.8335

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


87/87 [==============================] - 16s 177ms/step - loss: 0.4234 - accuracy: 0.8631 - val_loss: 0.3521 - val_accuracy: 0.9018
Epoch 2/50
87/87 [==============================] - 15s 173ms/step - loss: 0.1707 - accuracy: 0.9471 - val_loss: 0.1600 - val_accuracy: 0.9494
Epoch 3/50
87/87 [==============================] - 15s 172ms/step - loss: 0.1454 - accuracy: 0.9602 - val_loss: 0.1566 - val_accuracy: 0.9554
Epoch 4/50
87/87 [==============================] - 15s 176ms/step - loss: 0.1344 - accuracy: 0.9667 - val_loss: 0.5009 - val_accuracy: 0.9107
Epoch 5/50
87/87 [==============================] - 15s 173ms/step - loss: 0.1670 - accuracy: 0.9645 - val_loss: 0.0683 - val_accuracy: 0.9702
Epoch 6/50
87/87 [==============================] - 15s 173ms/step - loss: 0.1444 - accuracy: 0.9696 - val_loss: 0.1303 - val_accuracy: 0.9643
Epoch 7/50
87/87 [==============================] - 15s 171ms/step - loss: 0.0915 - accuracy: 0.9841 - val_loss: 0.0153 - val_accuracy: 0.9940
Epoch 8/50

In [ ]:
# Saving the model
model.save('best_model.h5')